In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%tensorflow_version 1.x
import tensorflow

TensorFlow 1.x selected.


In [3]:
import glob
import numpy as np
import collections
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [0]:
def test_network(test_images, mod, mod1):
    num_classes = 5
    arr = []
    arr2 = []
    for test_image in test_images:
      imgg = image.load_img(test_image, target_size=(224,224))
      x = image.img_to_array(imgg)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
      # ll =(mod.predict([np.array(x)]))
      ll = mod.predict(x)
      # ll1 = mod1.predict(x)
      # ll[0] = np.add(ll[0],ll1[0])
      # arr.append((test_image,np.argmax((ll[0]))))
      temp = np.argmax((ll[0]))
      arr.append(np.argmax((ll[0])))
      ll[0][temp] = 0
      arr2.append(np.argmax(ll[0]))

    # print(arr)
    z = np.array(arr)
    print(collections.Counter(z))
    return arr, arr2

In [8]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
VAL_DIR = "/content/drive/My Drive/nncomp/validation"
HEIGHT = 224
WIDTH = 224
val_generator = datagen.flow_from_directory(
    VAL_DIR, 
    target_size=(HEIGHT, WIDTH),
    subset='training'
)

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
TRAIN_DIR = "/content/drive/My Drive/nncomp/train"

train_generator = datagen.flow_from_directory(
    TRAIN_DIR, 
    target_size=(HEIGHT, WIDTH),
    subset='training'
)

Found 250 images belonging to 5 classes.
Found 2250 images belonging to 5 classes.


In [12]:
NUM_EPOCHS = 10
num_train_images = 1350
FC_LAYERS = [1024, 1024]
dropout = 0.5
adam = Adam(lr=0.00001)

base_model = VGG16(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=5)

finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
filepath="/content/drive/My Drive/finaldata/" + "mimic1" + ".h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

In [0]:
history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=250, 
                                       shuffle=True,callbacks = callbacks_list, verbose = 1, validation_data = val_generator)

In [0]:
d5 = sorted(glob.glob("/content/drive/My Drive/TestData/*.jpg"))
finetune_model.load_weights("/content/drive/My Drive/nncomp/60percent.h5")
finetune_model1.load_weights("/content/drive/My Drive/nncomp/onetwothree.h5")

In [33]:
m,n= test_network(d5, finetune_model1, finetune_model1)

Counter({0: 105, 4: 57, 2: 21, 1: 16, 3: 1})


In [35]:
from sklearn import metrics
print(metrics.confusion_matrix(x, ww))
# Print the precision and recall, among other metrics
print(metrics.classification_report(x, ww, digits=3))

[[50  6  2  0  0]
 [21 14  1  1  0]
 [11  9  7  5  2]
 [ 3  4  5  4  8]
 [ 0  0  0  0 47]]
              precision    recall  f1-score   support

           0      0.588     0.862     0.699        58
           1      0.424     0.378     0.400        37
           2      0.467     0.206     0.286        34
           3      0.400     0.167     0.235        24
           4      0.825     1.000     0.904        47

    accuracy                          0.610       200
   macro avg      0.541     0.523     0.505       200
weighted avg      0.570     0.610     0.566       200



In [14]:
from keras.utils.vis_utils import plot_model
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
x = [0, 3, 3, 0, 1, 4, 1, 0, 0, 0, 2, 4, 4, 2, 0, 1, 1, 2, 3, 0, 1, 0, 1, 0, 1, 0, 3, 0, 2, 0, 4, 1, 4, 2, 3, 4, 4, 1, 1, 0, 2, 4, 3, 0, 0, 4, 3, 2, 1, 1, 0, 0, 2, 4, 4, 4, 1, 4, 3, 1, 0, 0, 1, 2, 1, 2, 3, 1, 0, 4, 4, 1, 1, 4, 0, 4, 4, 3, 3, 2, 0, 0, 4, 0, 0, 4, 0, 0, 0, 4, 4, 2, 3, 2, 2, 1, 0, 4, 0, 2, 3, 4, 4, 0, 0, 2, 4, 0, 1, 1, 2, 1, 4, 1, 4, 3, 2, 1, 1, 4, 1, 0, 0, 4, 3, 2, 4, 4, 3, 2, 2, 4, 1, 2, 0, 0, 0, 1, 0, 2, 2, 3, 0, 1, 0, 3, 2, 1, 4, 4, 0, 1, 0, 2, 4, 3, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 4, 4, 4, 0, 4, 3, 1, 2, 3, 3, 2, 4, 4, 4, 3, 1, 4, 1, 2, 2, 0, 2, 3, 0, 2, 2, 4, 0, 4, 0, 4, 1, 0, 0]
# # z = [0, 4, 4, 0, 2, 4, 0, 0, 1, 0, 2, 4, 4, 2, 0, 0, 1, 2, 2, 0, 1, 0, 2, 0, 3, 0, 4, 0, 2, 0, 4, 0, 4, 0, 2, 4, 4, 0, 1, 0, 2, 4, 2, 0, 0, 4, 0, 2, 1, 0, 0, 0, 1, 4, 4, 4, 2, 4, 2, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 4, 4, 1, 1, 4, 0, 4, 4, 1, 0, 0, 0, 0, 4, 0, 1, 4, 0, 0, 0, 4, 4, 1, 0, 1, 3, 0, 0, 4, 0, 0, 4, 4, 4, 0, 0, 3, 4, 0, 1, 1, 2, 0, 4, 0, 4, 4, 1, 0, 0, 4, 1, 0, 2, 4, 3, 0, 4, 4, 2, 2, 0, 4, 0, 2, 2, 0, 0, 0, 2, 0, 1, 4, 0, 0, 0, 1, 0, 1, 4, 4, 0, 1, 0, 0, 4, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 4, 4, 4, 0, 4, 4, 0, 1, 4, 0, 0, 4, 4, 4, 1, 0, 4, 0, 0, 1, 0, 0, 4, 0, 1, 1, 4, 1, 4, 0, 4, 0, 0, 0]
# # y = [0, 4, 4, 0, 1, 4, 0, 0, 1, 0, 2, 4, 4, 3, 2, 4, 1, 0, 2, 0, 1, 0, 0, 0, 3, 0, 4, 0, 0, 0, 4, 0, 4, 0, 3, 4, 4, 0, 1, 0, 3, 4, 4, 0, 0, 4, 0, 0, 1, 0, 0, 0, 1, 4, 4, 4, 0, 4, 4, 0, 1, 0, 1, 0, 0, 4, 2, 0, 0, 4, 4, 1, 1, 4, 0, 4, 4, 1, 0, 0, 0, 0, 4, 0, 1, 4, 0, 0, 0, 4, 4, 2, 1, 1, 4, 0, 0, 4, 0, 0, 4, 4, 4, 0, 0, 4, 4, 0, 1, 1, 2, 0, 4, 0, 4, 4, 3, 0, 1, 4, 1, 0, 2, 4, 2, 0, 4, 4, 4, 4, 2, 4, 0, 4, 0, 0, 0, 0, 0, 2, 4, 4, 0, 0, 0, 1, 0, 1, 4, 4, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 4, 4, 0, 1, 4, 0, 0, 4, 4, 4, 4, 0, 4, 0, 0, 1, 0, 0, 3, 0, 1, 1, 4, 0, 4, 0, 4, 0, 1, 0]
# q = [0, 4, 4, 0, 1, 4, 0, 0, 1, 0, 2, 4, 4, 3, 2, 4, 1, 0, 2, 0, 1, 0, 0, 0, 3, 0, 4, 0, 0, 0, 4, 0, 4, 0, 3, 4, 4, 0, 1, 0, 3, 4, 4, 0, 0, 4, 0, 0, 1, 0, 0, 0, 1, 4, 4, 4, 0, 4, 4, 0, 1, 0, 1, 0, 0, 4, 2, 0, 0, 4, 4, 1, 1, 4, 0, 4, 4, 1, 0, 0, 0, 0, 4, 0, 1, 4, 0, 0, 0, 4, 4, 2, 1, 1, 4, 0, 0, 4, 0, 0, 4, 4, 4, 0, 0, 4, 4, 0, 1, 1, 2, 0, 4, 0, 4, 4, 3, 0, 1, 4, 1, 0, 2, 4, 2, 0, 4, 4, 4, 4, 2, 4, 0, 4, 0, 0, 0, 0, 0, 2, 4, 4, 0, 0, 0, 1, 0, 1, 4, 4, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 4, 4, 0, 1, 4, 0, 0, 4, 4, 4, 4, 0, 4, 0, 0, 1, 0, 0, 3, 0, 1, 1, 4, 0, 4, 0, 4, 0, 1, 0]


In [19]:
print(collections.Counter(x))
print(collections.Counter(r))
print(collections.Counter(q))

Counter({0: 58, 4: 47, 1: 37, 2: 34, 3: 24})
Counter({3: 74, 2: 55, 1: 36, 0: 28, 4: 7})
Counter({0: 91, 4: 66, 1: 27, 2: 10, 3: 6})


In [42]:
count = 0
for i,j,k in zip(x,m,n):
  if i in [j,k]:
    count += 1
print(count/200)

0.765


In [34]:
count = 0
ww = []
for i,j,k in zip(x,m,q):#q-60%, y-mix
  t = k if k in [1,2,3] else int((j+k)/2)
  if i == t:
    count += 1
  ww.append(t)
print(count)
print(collections.Counter(q)) #basic classifier
print(collections.Counter(m)) #priority classifier
print(collections.Counter(ww))#ensemble classifier

122
Counter({0: 91, 4: 66, 1: 27, 2: 10, 3: 6})
Counter({0: 105, 4: 57, 2: 21, 1: 16, 3: 1})
Counter({0: 85, 4: 57, 1: 33, 2: 15, 3: 10})
